# Working with a database

This week's assignment has a few basic steps.  First, we're going to pull some data down off the internet and store it into our MySQL database.  Make sure that you use your username as part of the table name as show in the examples so that you don't create a problem for other students.

Then, we'll merge that with some data already in the database and calculate a few results.  When it comes to calculating the results, you can do so either with SQL or with Pandas operations.


## PART 1: Setup your database connection and table name

In the code below, change the value of the variable `MYTABLE` to use your own username rather that `'pboal'`

You can then use `MYTABLE` in the rest of your code to reference that table name.

In [1]:
import getpass
myname = getpass.getuser().split('-')[1]
myname
MYTABLE=myname + '_data'

print(MYTABLE)

rasalt_data


In [80]:
import os
from sqlalchemy import create_engine
import pymysql
import pandas as pd

host = 'slucor2022-instance-1.cgdcoitnku0k.us-east-1.rds.amazonaws.com'
port = '3306'
user = 'slucor2022'
password = 'SLUcor2022'
database = 'hds5210'
conn = create_engine('mysql+pymysql://' + 
                     user + ':' + 
                     password + '@' + 
                     host + '/' + 
                     database, echo=False)

In [81]:
assert(MYTABLE != 'pboal')
assert(conn.connect())

## PART 2: Bring in outside data

Grab the data from this URL and put it into a database table named with your `username_data`.

https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326

By the end of your cell, the table should be created.  The tests are going to verify that the table exists and looks right.

In [4]:
url = "https://opendata.arcgis.com/api/v3/datasets/ed0bc689bde246b18835c7529ba4efb4_0/downloads/data?format=csv&spatialRefId=4326"
df = pd.read_csv(url)
df.head()

,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,Business_ZIP_Code,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,95501,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,94545,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,94558,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,95642,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,94114,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019


In [5]:
df.to_sql(MYTABLE, conn, if_exists='replace')
dfnew = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
dfnew.shape

(323, 22)

In [7]:
dxyz = pd.read_sql_query('SELECT * FROM ' + MYTABLE, conn)
dxyz.head()

,index,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,...,Payment_Year_Number,Program_Type,Medicaid_EP_Hospital_Type,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year
0,0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,...,4,Medicare/Medicaid,Acute Care Hospitals,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016
1,1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,...,4,Medicare/Medicaid,Acute Care Hospitals,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016
2,2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,...,4,Medicare/Medicaid,Acute Care Hospitals,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016
3,3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,...,4,Medicare/Medicaid,Acute Care Hospitals,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019
4,4,-122.435820,37.769049,5,Sutter Bay Hospitals,1659439834,50008,CASTRO AND DUBOCE,San Francisco,San Francisco,...,4,Medicare/Medicaid,Acute Care Hospitals,984465.12,MU,98446.51,37.769049,-122.435820,2016,2019


In [8]:

assert(dxyz.shape == (323,22))
assert(list(dxyz.columns) == ['index', 'X', 'Y', 'OBJECTID', 'Provider_Name', 'NPI', 'CCN',
       'Business_Street_Address', 'Business_City', 'Business_County',
       'Business_ZIP_Code', 'Business_State_Territory', 'Payment_Year_Number',
       'Program_Type', 'Medicaid_EP_Hospital_Type', 'total_payments',
       'Last_Payment_Criteria', 'Recent_Disbursement_Amount', 'Latitude',
       'Longitude', 'Last_Program_Year', 'Last_Payment_Year'])

## PART 3: Combine with other data in the database

In the database, there is an existing table called `population`.  We want to merge the DHCS datafile loaded above with the population data available in this other database table  The tables can be merged on `MYTABLE`'s `Business_ZIP_Code` field and `population`'s `Zip` field.

Note that not all `Zip_Codes` from your downloaded file have to be in the `population` table.  If they aren't, then I want you to eliminate the non-matching records.  That is, only keep the records that have a matching ZIP code in both sets of data.

Answer the question:
Which providers are located in the zipcode with the largest population?

Put your answer in the form `answer = ['a', 'list', 'of', 'NPI', 'like', '1593042103]`

In [112]:
dabc = pd.read_sql_query('SELECT * FROM ' + "population", conn)
dabc.shape
dabc.value_counts().value_counts()

index  Zip    Population
1649   92364  5             1
553    92583  32077         1
543    94545  32552         1
544    91006  32526         1
545    92373  32433         1
                           ..
1102   92590  5215          1
1103   90290  5200          1
1104   96101  5152          1
1105   92210  5137          1
0      90011  110750        1
Length: 1650, dtype: int64

In [10]:
dabc.head()

,index,Zip,Population
0,0,90011,110750
1,1,90650,104524
2,2,91331,103683
3,3,90201,100512
4,4,90044,99980


In [11]:
djoin = dxyz.merge(dabc, how="inner", left_on="Business_ZIP_Code", right_on="Zip")
djoin.shape

(317, 25)

In [12]:
djoin.head()

,index_x,X,Y,OBJECTID,Provider_Name,NPI,CCN,Business_Street_Address,Business_City,Business_County,...,total_payments,Last_Payment_Criteria,Recent_Disbursement_Amount,Latitude,Longitude,Last_Program_Year,Last_Payment_Year,index_y,Zip,Population
0,0,-124.142009,40.783559,1,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1609858950,50006,2700 Dolbeer St,Eureka,Humboldt,...,1530950.70,MU,153095.07,40.783559,-124.142009,2015,2016,736,95501,23948
1,1,-122.086674,37.632915,2,HAYWARD SISTERS HOSPITAL,1942298153,50002,27200 Calaroga Ave,Hayward,Alameda,...,3245920.28,MU,324592.03,37.632915,-122.086674,2015,2016,543,94545,32552
2,194,-122.096165,37.630272,195,KAISER FOUNDATION HOSPITALS,1801960513,50512,27400 Hesperian Blvd.,Hayward,Alameda,...,415091.50,MU,41509.15,37.630272,-122.096165,2018,2020,543,94545,32552
3,2,-122.295861,38.325402,3,ST JOSEPH HEALTH NORTHERN CALIFORNIA LLC,1235218785,50009,1000 Trancas St,Napa,Napa,...,1262015.89,MU,126201.59,38.325402,-122.295861,2015,2016,77,94558,66854
4,3,-120.766467,38.350920,4,SUTTER VALLEY HOSPITALS,1447494323,50014,200 Mission Blvd,Jackson,Amador,...,865130.13,MU,86513.01,38.350920,-120.766467,2017,2019,1067,95642,6547


In [13]:
#djoin.head()
# Zip with the largest population
maxzip = djoin[djoin['Population'] == djoin['Population'].max()]['Zip'].values[0]
print(maxzip)

90650


In [14]:
providers = djoin[djoin['Zip']==maxzip]['NPI'].astype(str)
answer = providers.tolist()
providers
print(answer)

['1194016923']


In [15]:
assert(type(answer) == list)
assert(answer == ['1194016923'])

## PART 4: Total by hospital type

This is a multistep process:
* From our downloaded data file, compute the `Total payments` per ZIP code and Medicaid EP Hospital Type.
* Then merge that with the `population` data to compute a `Total payments` per person.
* Then average that across all of the `Medicaid EP Hospital Types` to get an average per persona payment for these type of hospital.

Your answer should be in structure of a data frame with at least two columns:
* Medicaid_EP_Hospital_Type
* Avg_Pay_per_Capita

In [82]:
MYTABLE

'rasalt_data'

## First using SQL

In [113]:
MYTABLE_TMP1 = MYTABLE+"_tmp1"
MYTABLE_TMP2 = MYTABLE+"_tmp2"

df1 = pd.read_sql_query("""SELECT Medicaid_EP_Hospital_Type,Business_ZIP_Code, sum(total_payments) as Total FROM {} GROUP BY Business_ZIP_Code, Medicaid_EP_Hospital_Type"""
                        .format(MYTABLE), conn)
df1.to_sql(MYTABLE_TMP1, conn, if_exists='replace')
df1.head()
#df1['Business_ZIP_Code'].value_counts()

,Medicaid_EP_Hospital_Type,Business_ZIP_Code,Total
0,Acute Care Hospitals,90004,1093021.73
1,Acute Care Hospitals,90012,3614739.93
2,Acute Care Hospitals,90015,10045484.04
3,Acute Care Hospitals,90017,3899845.21
4,Children's Hospitals,90020,427640.22


In [114]:
df2 = pd.read_sql_query("""
        SELECT t1.*, t2.Population
        FROM 
          {} t1 JOIN
          population t2 ON t1.Business_ZIP_Code = t2.Zip""".format(MYTABLE_TMP1), conn)
df2['per_person_zip'] = df2['Total']/df2['Population']
df2.to_sql(MYTABLE_TMP2, conn, if_exists='replace')
df2.head()
#df2['Medicaid_EP_Hospital_Type'].value_counts()

,index,Medicaid_EP_Hospital_Type,Business_ZIP_Code,Total,Population,per_person_zip
0,33,Acute Care Hospitals,90650,2184880.10,104524,20.903143
1,97,Acute Care Hospitals,92335,1108375.17,99242,11.168408
2,47,Acute Care Hospitals,91342,9246604.47,94432,97.918126
3,106,Acute Care Hospitals,92503,4212811.78,93032,45.283470
4,60,Acute Care Hospitals,91710,1926048.03,90481,21.286768


In [117]:
answer = pd.read_sql_query("""SELECT Medicaid_EP_Hospital_Type, avg(per_person_zip) as 'Avg_Pay_per_Capita' FROM {} GROUP BY Medicaid_EP_Hospital_Type""".format(MYTABLE_TMP2), conn)

In [118]:
answer.head()

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,329.550946


In [119]:
assert(type(answer) == pd.core.frame.DataFrame)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Acute Care Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 106.583)
assert(round(answer.query("Medicaid_EP_Hospital_Type == 'Children\\'s Hospitals'")['Avg_Pay_per_Capita'].sum(),3) == 329.551)


In [120]:
answer

,Medicaid_EP_Hospital_Type,Avg_Pay_per_Capita
0,Acute Care Hospitals,106.582669
1,Children's Hospitals,329.550946


## Now using Pandas instead of SQL

In [133]:
dxyz.head()
dxyz['Business_ZIP_Code']

0      95501
1      94545
2      94558
3      95642
4      94114
       ...  
318    94304
319    90806
320    95817
321    90020
322    92123
Name: Business_ZIP_Code, Length: 323, dtype: int64

In [142]:
answer = dxyz.groupby(['Business_ZIP_Code', 'Medicaid_EP_Hospital_Type'],as_index=False)['total_payments'].sum()

In [143]:
answer = answer.merge(dabc, how="inner", left_on="Business_ZIP_Code", right_on="Zip")
answer['per_person_zip'] = answer['total_payments']/answer['Population']
answer

,Business_ZIP_Code,Medicaid_EP_Hospital_Type,total_payments,index,Zip,Population,per_person_zip
0,90004,Acute Care Hospitals,1093021.73,115,90004,61105,17.887599
1,90012,Acute Care Hospitals,3614739.93,433,90012,37268,96.993129
2,90015,Acute Care Hospitals,10045484.04,738,90015,23900,420.313140
3,90017,Acute Care Hospitals,3899845.21,648,90017,27832,140.120912
4,90020,Children's Hospitals,427640.22,390,90020,39366,10.863187
...,...,...,...,...,...,...,...
256,96104,Acute Care Hospitals,184615.39,1388,96104,920,200.668902
257,96122,Acute Care Hospitals,531907.08,1169,96122,3594,147.998631
258,96130,Acute Care Hospitals,1371962.88,810,96130,19859,69.085195
259,96150,Acute Care Hospitals,786769.13,595,96150,30369,25.906982


In [145]:
answer.groupby('Medicaid_EP_Hospital_Type')['per_person_zip'].mean()
#answer.head()

Medicaid_EP_Hospital_Type
Acute Care Hospitals    106.582669
Children's Hospitals    329.550946
Name: per_person_zip, dtype: float64



---



## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [65]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git pull
    !git add week15_assignment.ipynb
    !git commit -a -m "Submitting the week 15 programming assignment"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


Already up to date.
[main 0c57349] Submitting the week 15 programming assignment
 2 files changed, 1908 insertions(+), 2 deletions(-)
 create mode 100644 week15/week15_assignment.ipynb
Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 8.68 KiB | 2.89 MiB/s, done.
Total 6 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 2 local objects.
To github.com:rasalt/hds5210-2022.git
   bf1e48b..0c57349  main -> main
